# 1. 什么是 model.save()？
`model.save()` 是 `TensorFlow/Keras` 模型对象的一个方法，用于将整个模型保存到磁盘。保存的内容包括：

- 1、模型的架构 (Model Architecture)：每一层是什么以及它们如何连接。

- 2、模型的权重 (Model Weights)：训练得到的参数。

- 3、模型的训练配置 (编译信息)：优化器、损失函数、指标等（如果模型已经编译过）。

- 4、优化器的状态 (Optimizer State)：允许你从上次中断的地方重新开始训练。

简单来说，`model.save()` 为你提供了一个“模型快照”，你之后可以完整地恢复这个快照，直接用于继续训练或进行预测，而无需重新定义和编译模型。

---
# 2. 为什么要使用 `model.save()`？
- 1、模型持久化：将训练好的模型保存下来，供后续使用，无需重新训练。

- 2、分享模型：方便将模型分享给团队成员或部署到生产环境。

- 3、继续训练：如果训练过程意外中断，可以从保存的点继续训练，而不是从头开始。

- 4、部署推理：将模型交给 Web 服务、移动设备或其他应用程序进行预测。

---
## 3. 两种保存格式
从 `TensorFlow 2.4` 开始，`model.save()` 主要支持两种格式：

### (A) TensorFlow SavedModel 格式（默认）: 
这是 `TensorFlow` 生态系统的官方推荐格式。它是一个包含完整 `TensorFlow` 程序的目录。

- 特点：

    - 包含所有模型信息，可被 `TensorFlow Serving`、`TensorFlow Lite`、`TensorFlow.js` 等工具直接使用。
    
    - 具有版本控制，兼容未来的 `TensorFlow` 版本。
    
    - 是 `model.save()` 的默认格式。

保存方法：

In [ ]:
# 方法 1: 使用默认的 SavedModel 格式
model.save('my_model') # 注意：传入的是目录路径，不是文件名

# 方法 2: 显式指定格式
model.save('my_model', save_format='tf')

# 加载
model = tf.keras.models.load_model("my_model")

执行后，会创建一个名为 `my_model` 的文件夹，里面包含以下文件：

In [ ]:
my_model/
├── assets/
├── variables/
│   ├── variables.data-00000-of-00001
│   └── variables.index
├── keras_metadata.pb
└── saved_model.pb

- assets/: 存储模型所需的其他文件（如词汇表）。
- variables/: 存储模型的权重。
- keras_metadata.pb: 存储 Keras 特定的元数据。
- saved_model.pb: 存储模型架构和训练配置。

---
### (B) Keras `H5` 格式 \
这是一种传统的、基于单个 `HDF5` 文件的格式。

- 特点：

    - 所有内容（架构、权重、配置）都保存在一个 `.h5` 或 `.keras` 文件中。
    
    - 便于管理和分享（只有一个文件）。
    
    - 在某些环境下可能不如 `SavedModel` 格式兼容性好。

保存方法：

In [ ]:
# 显式指定保存为 .h5 文件
model.save('my_model.h5') # 在文件名中包含 .h5 扩展名

# 或者显式使用 save_format 参数
model.save('my_model.keras', save_format='h5') # TF 2.13+ 推荐 .keras 扩展名

## 4. 如何加载保存的模型？
使用 `tf.keras.models.load_model()` 函数来加载保存的模型。`TensorFlow` 会自动识别格式。

加载 `SavedModel` 格式:

In [ ]:
# 加载 SavedModel 格式的模型（从目录加载）
loaded_model = tf.keras.models.load_model('my_model')

加载 H5 格式

In [ ]:
# 加载 H5 格式的模型（从文件加载）
loaded_model = tf.keras.models.load_model('my_model.h5')

### 加载后的模型与原始模型完全相同，可以直接使用：

In [ ]:
# 进行预测
predictions = loaded_model.predict(new_data)

# 查看模型摘要
loaded_model.summary()

# 甚至可以继续训练（如果保存时包含了优化器状态）
# loaded_model.fit(more_data, more_labels, epochs=10)

## 5. 完整工作流程示例
让我们通过一个完整的例子来演示保存和加载的流程。

In [ ]:
import tensorflow as tf
import numpy as np

# 1. 构建并训练一个简单的模型
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 生成一些虚拟数据
x_train = np.random.random((1000, 10))
y_train = np.random.randint(2, size=(1000, 1))

# 训练模型
model.fit(x_train, y_train, epochs=5, verbose=1)

# 2. 保存模型（两种格式）
# 保存为 SavedModel 格式（默认）
model.save('my_saved_model') 

# 保存为 H5 格式
model.save('my_model.h5')    

print("模型已保存！")

# 3. 加载模型并测试
# 加载 SavedModel
loaded_savedmodel = tf.keras.models.load_model('my_saved_model')

# 加载 H5 模型
loaded_h5_model = tf.keras.models.load_model('my_model.h5')

# 生成一些测试数据
test_data = np.random.random((5, 10))

# 使用原始模型预测
original_pred = model.predict(test_data)
print("原始模型预测:", original_pred.flatten())

# 使用加载的模型预测（结果应完全相同）
savedmodel_pred = loaded_savedmodel.predict(test_data)
h5_pred = loaded_h5_model.predict(test_data)

print("SavedModel 预测:", savedmodel_pred.flatten())
print("H5 模型预测:    ", h5_pred.flatten())

# 验证预测结果是否相同
print("预测结果是否相同:", np.allclose(original_pred, savedmodel_pred) and np.allclose(original_pred, h5_pred))

## 6. 两种格式如何选择？
| 特性             | SavedModel 格式 (默认)                          | H5 格式                          |
| :--------------- | :---------------------------------------------- | :------------------------------- |
| **文件数量**     | 一个目录，包含多个文件                          | 单个文件                         |
| **兼容性**       | TensorFlow 生态首选，被 TF Serving, Lite, .js 支持 | 主要适用于 Keras                 |
| **自定义对象**   | 支持较好，需在加载时传递 `custom_objects` 参数    | 支持较好，需在加载时传递 `custom_objects` 参数 |
| **版本控制**     | 有，面向未来兼容性更好                          | 较弱                             |
| **推荐场景**     | 生产环境、部署、继续训练                        | 研究、分享、快速实验             |

**一般建议：**

*   默认使用 `SavedModel` 格式（即不指定扩展名），除非你有特殊需求。
*   当你需要单个文件以便于邮件发送或管理时，使用 H5 格式（`.h5` 或 `.keras`）。

---
## 7. 处理自定义对象
如果你的模型包含自定义层、损失函数或指标，需要在加载时通过 `custom_objects` 参数告诉 `TensorFlow` 如何解析它们。

定义自定义层：

In [ ]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                initializer='random_normal',
                                trainable=True)
    def call(self, inputs):
        return tf.matmul(inputs, self.w)

    def get_config(self):
        return {'units': self.units}

In [ ]:
保存和加载包含自定义层的模型：

In [ ]:
# 创建并保存包含自定义层的模型
model = tf.keras.Sequential([CustomLayer(10), tf.keras.layers.Dense(1)])
model.compile(optimizer='adam', loss='mse')
model.save('custom_model') # 保存

# 加载时提供自定义对象的映射字典
loaded_model = tf.keras.models.load_model(
    'custom_model',
    custom_objects={'CustomLayer': CustomLayer} # 关键：告诉TF如何解析CustomLayer
)

# 总结
`model.save()` 是保存整个模型的首选方法。

默认格式是 `SavedModel`（目录），适用于生产和部署。

使用 `.h5` 或 `.keras` 扩展名来保存为单个 `H5` 文件，便于分享和研究。

使用 `tf.keras.models.load_model()` 来加载模型，对于自定义对象需要提供 `custom_objects` 参数字典。

保存的模型包含完整信息，可以用于预测或继续训练。